In [1]:
import xesmf as xe
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter as gauss
from matplotlib.colors import LogNorm
import xroms
import numpy as np
import seawater as sw
from dask.diagnostics import ProgressBar

import sys
sys.path.append("/homes/metogra/iufarias/Documents/Scripts/useful")
import romspickle as rp

from dask.diagnostics import ProgressBar
import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
chunks={'time_counter':40, 'x_rho':1000, 'y_rho':1520,'s_rho':40,'s_w':40}

#3D
path='/data/pacific/lrenault/SASI/CROCO/'

varf=xr.open_mfdataset(path + 'FULL/SASI*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*.nc',data_vars='minimal', chunks=chunks)



In [4]:
# zf=rp.zlevs(h=varf.h.to_numpy(),zeta=varf.zeta[0].to_numpy(),
#          theta_s=np.int(varf.theta_s),theta_b=np.int(varf.theta_b),
#          hc=np.int(varf.hc),N=80)

# zt=rp.zlevs(h=varf.h.to_numpy(),zeta=varf.zeta[0].to_numpy(),
#          theta_s=np.int(varf.theta_s),theta_b=np.int(varf.theta_b),
#          hc=np.int(varf.hc),N=80) 

In [5]:
# varf=varf.assign_coords(z_rho=(('s_rho','y_rho','x_rho'),zf))
# vart=vart.assign_coords(z_rho=(('s_rho','y_rho','x_rho'),zt))

In [6]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=2)


vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=2)

In [7]:
# tf=df.temp.groupby("ocean_time.dayofyear").mean();
# tf=tf[0,-15:-1,200:1350,20:850]
# # tf=tf.isel(s_rho=range(-15,-1),xi_rho=range(20,850));
# # tf=tf.isel(eta_rho=range(100,1300));
# # tf=tf.compute()

# sf=df.salt.groupby("ocean_time.dayofyear").mean();
# sf=sf[0,-15:-1,200:1350,20:850]

In [8]:
#rechunk
#groupby mean is breaking the chunks
# maybe use xrft in mor than 2d if specifiing the dims? 


In [14]:
chunks_rho={'dayofyear':14, 'xi_rho':1000, 'eta_rho':1520,'s_rho':40}
# chunks_rho={'month':1, 'xi_rho':1000, 'eta_rho':1520,'s_rho':8}


In [11]:
%%time
with ProgressBar():
    aa=df.temp[0,-1,:,:].compute()

[########################################] | 100% Completed |  7.1s
CPU times: user 7.32 s, sys: 4.69 s, total: 12 s
Wall time: 7.16 s


In [15]:
%%time

tf=df.temp.groupby("ocean_time.dayofyear").mean();
# tf=df.temp.groupby("ocean_time.month").mean();
tf=tf.isel(s_rho=range(-15,-1),xi_rho=range(800));
tf=tf.isel(eta_rho=range(100,1300));
tf=tf.chunk(chunks=chunks_rho)


sf=df.salt.groupby("ocean_time.dayofyear").mean();
# sf=df.salt.groupby("ocean_time.month").mean();
sf=sf.isel(s_rho=range(-15,-1),xi_rho=range(800));
sf=sf.isel(eta_rho=range(100,1300));
sf=sf.chunk(chunks=chunks_rho)

KeyboardInterrupt: 

In [10]:
%%time

rhof=xroms.potential_density(tf,sf)
bf=xroms.buoyancy(rhof)

CPU times: user 137 ms, sys: 2.97 ms, total: 140 ms
Wall time: 138 ms


In [11]:
%%time

tt=dt.temp.groupby("ocean_time.dayofyear").mean();
# tt=dt.temp.groupby("ocean_time.month").mean();
tt=tt.isel(s_rho=range(-15,-1),xi_rho=range(800));
tt=tt.isel(eta_rho=range(100,1300));
tt=tt.chunk(chunks=chunks_rho)


st=dt.salt.groupby("ocean_time.dayofyear").mean();
# st=dt.salt.groupby("ocean_time.month").mean();
st=st.isel(s_rho=range(-15,-1),xi_rho=range(800));
st=st.isel(eta_rho=range(100,1300));
st=st.chunk(chunks=chunks_rho)





CPU times: user 94.7 ms, sys: 3.49 ms, total: 98.2 ms
Wall time: 97.1 ms


In [12]:
%%time
rhot=xroms.potential_density(tt,st)
bt=xroms.buoyancy(rhot)

CPU times: user 138 ms, sys: 2.51 ms, total: 140 ms
Wall time: 138 ms


In [13]:
%%time 
wf=df.w.groupby("ocean_time.dayofyear").mean();
# wf=df.w.groupby("ocean_time.month").mean();
wf=wf.isel(s_rho=range(-15,-1),xi_rho=range(800));
wf=wf.isel(eta_rho=range(100,1300));
wf=wf.chunk(chunks=chunks_rho)




CPU times: user 49.6 ms, sys: 517 µs, total: 50.1 ms
Wall time: 49.3 ms


In [14]:
%%time
wt=dt.w.groupby("ocean_time.dayofyear").mean();
wt=dt.w.groupby("ocean_time.month").mean();
wt=wt.isel(s_rho=range(-15,-1),xi_rho=range(800));
wt=wt.isel(eta_rho=range(100,1300));
wt=wt.chunk(chunks=chunks_rho)




CPU times: user 49.6 ms, sys: 3.47 ms, total: 53 ms
Wall time: 52 ms


In [15]:
bbb=wt[:,-1,:,:]

In [16]:
bbb=bbb.chunk({'month':1})

In [18]:
%%time
bbb=bbb.compute()

KeyboardInterrupt: 

In [25]:
%%time
exspec=xrft.isotropic_power_spectrum(bbb.drop(['nav_lat_rho', 'nav_lon_rho', 'z_rho0']).compute(),dim=['xi_rho','eta_rho'],nfactor=2,truncate='True',detrend='linear', window=True)

KeyboardInterrupt: 

In [71]:
%%time
VBPf=xrft.isotropic_cross_spectrum(wf.drop(['nav_lat_rho', 'nav_lon_rho', 'z_rho0'])
                                     ,bf.drop(['nav_lat_rho', 'nav_lon_rho', 'z_rho0']),dim=['xi_rho','eta_rho'],nfactor=2,truncate='True',detrend='linear', window=True)

KeyboardInterrupt: 

In [51]:
exspec=xrft.isotropic_power_spectrum(bf[0,0,:,:],nfactor=2,truncate='True',detrend='linear', window=True)

VBPf=np.zeros([Bf.shape[0],Bf.shape[1],exspec.shape[0]])*np.nan;
VBPt=np.zeros([Bt.shape[0],Bt.shape[1],exspec.shape[0]])*np.nan;

for idd in range(Bf.shape[0]):
    for izz in range(Bf.shape[1]):
        print(ixx,end=' ',flush=True)

        bbf=xrft.isotropic_power_spectrum(uf[idd,izz,:,:],nfactor=2,truncate='True',detrend='linear', window=True)

        bbt=xrft.isotropic_power_spectrum(ut[idd,izz,:,:],nfactor=2,truncate='True',detrend='linear', window=True)



/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/xrft/xrft.py:51: FutureWarning: Please provide the name of window adhering to scipy.signal.windows. The boolean option will be deprecated in future releases.
  FutureWarning,


### Mixed Layer Depth